In [ ]:
# FeatureSlider.ipynb — minimal, function-based dashboard for JupyterLite/Voici

from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W

# -----------------------------
# 1) Data loading utilities
# -----------------------------
def _try_read_csv(paths: list[str]) -> tuple[pd.DataFrame, str]:
    """
    Try a list of candidate paths and return (df, resolved_path).
    Works both in JupyterLite builds and local runs.
    """
    last_err = None
    for p in paths:
        try:
            df = pd.read_csv(p)
            return df, p
        except Exception as e:
            last_err = e
    raise RuntimeError(f"Could not load meta.csv from any of: {paths}\nLast error: {last_err}")

def load_meta() -> tuple[pd.DataFrame, str]:
    candidates = [
        "data/meta.csv",            # when this notebook sits in content/
        "content/data/meta.csv",    # if someone runs it from repo root
        "/files/data/meta.csv",     # JupyterLite local FS (rarely needed)
        "/drive/data/meta.csv",     # Some cloud/lite mounts
    ]
    return _try_read_csv(candidates)

# -----------------------------
# 2) Column detection helpers
# -----------------------------
def detect_columns(df: pd.DataFrame) -> tuple[str | None, list[str]]:
    """
    Heuristically find a unit column (if any) and three feature columns.
    Prefers conventional names; otherwise picks the first three numeric cols.
    """
    # likely unit id column
    unit_candidates = ["unit", "unit_id", "Unit", "id", "neuron_id"]
    unit_col = next((c for c in unit_candidates if c in df.columns), None)

    # preferred feature name sets
    preferred_sets = [
        ["feature1", "feature2", "feature3"],
        ["Feature1", "Feature2", "Feature3"],
        ["feature_1", "feature_2", "feature_3"],
        ["feat1", "feat2", "feat3"],
        ["F1", "F2", "F3"],
    ]
    for s in preferred_sets:
        if all(c in df.columns for c in s):
            return unit_col, s

    # fallback: first three numeric columns (excluding unit_col)
    numeric_cols = [c for c in df.columns
                    if pd.api.types.is_numeric_dtype(df[c]) and c != unit_col]
    if len(numeric_cols) < 3:
        raise ValueError(
            f"Need at least 3 numeric feature columns; found {len(numeric_cols)}: {numeric_cols}"
        )
    return unit_col, numeric_cols[:3]

# -----------------------------
# 3) Plotting
# -----------------------------
def plot_feature(df: pd.DataFrame, feature_cols: list[str], feature_index: int, unit_col: str | None = None):
    """
    Plot feature N (1-based) across all units.
    """
    idx = int(feature_index) - 1
    if not (0 <= idx < len(feature_cols)):
        raise IndexError(f"feature_index {feature_index} out of range for {len(feature_cols)} features.")
    col = feature_cols[idx]
    y = df[col].to_numpy()
    x = np.arange(len(y))

    plt.figure(figsize=(7.2, 4.5))
    plt.plot(x, y, marker="o", linestyle="-")
    plt.title(f"{col} across all units (n={len(y)})")
    plt.xlabel(unit_col if unit_col else "Unit index")
    plt.ylabel(col)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# -----------------------------
# 4) Build the small UI
# -----------------------------
def make_app():
    df, resolved = load_meta()
    unit_col, feature_cols = detect_columns(df)

    info = W.HTML(
        f"<b>Loaded:</b> <code>{resolved}</code><br>"
        f"<b>Feature columns:</b> {', '.join(map(str, feature_cols))}"
    )

    slider = W.IntSlider(
        value=1, min=1, max=3, step=1,
        description="Feature",
        continuous_update=False
    )

    out = W.Output()

    def render(*_):
        with out:
            out.clear_output(wait=True)
            plot_feature(df, feature_cols, slider.value, unit_col)

    slider.observe(render, names="value")
    render()  # initial draw
    return W.VBox([info, slider, out], layout=W.Layout(width="900px"))

app = make_app()
display(app)
